## ① フルコンプ買取データのアップデート

In [1]:
#ライブラリのインポート

import pandas as pd
import requests
from bs4 import BeautifulSoup

# 買取データの日付を取得する。
import datetime 
from datetime import date, timedelta
today = datetime.date.today()
today = today.strftime('%y-%m-%d')

#表示の設定を行う。
pd.set_option('display.max_columns',10)

In [2]:
#　requestsメソッドを使って、サイトからHTMLの情報を抽出する。
url = 'https://www.fullcomp.jp/radiokaikan/kaitori/19439'
res = requests.get(url)

In [3]:
#　BeautifulSoup()の設定を行う。
soup = BeautifulSoup(res.content,'html.parser')

In [4]:
# 価格情報を抽出する。
soup_data = soup.find('tbody',class_ = 'fs-small')
card_data = soup_data.find_all('td') 

In [5]:
#情報をリストに格納する。
card_rare = []
card_name = []
card_price = []

data_len = len(card_data) - 1
data_len/3

for i in range(int(data_len/3)):
    card_rare.append(card_data[3*i].string)
    card_name.append(card_data[3*i+1].string)
    card_price.append(card_data[3*i+2].string)
    
# データの結合を行う。
df_norm = pd.concat([
    pd.DataFrame(card_name),
    pd.DataFrame(card_rare),
    pd.DataFrame(card_price)],axis = 1)


#カラムを設定する。
df_norm.columns = ['カード名','R',today]

In [6]:
#エラー表示の停止、表示条件の記載を行う。
import warnings
warnings.filterwarnings('ignore')
len_num = len(df_norm)
pd.set_option('display.max_rows',10)
print('加工前の列数:',len(df_norm))

#【】のない行に【】を追記する。
df_norm_split = df_norm[~df_norm['カード名'].str.contains('【')]
df_norm_split['カード名'] = '【_】' + df_norm[~df_norm['カード名'].str.contains('【')]['カード名']
df_norm = pd.concat([df_norm[df_norm['カード名'].str.contains('【')],df_norm_split])

print('加工後の列数:',len(df_norm))
df_norm.sort_index(inplace = True)

加工前の列数: 1985
加工後の列数: 1985


In [7]:
#カラムを設定する。
df_norm.columns = [0,1,2]
pd.set_option('display.max_rows',10)

# #文字列の分割を行う。
df_norm_split = df_norm[0].str.split('【',expand = True)
df_norm_split = df_norm_split[1].str.split('】',expand = True)
df_norm_split = df_norm_split[1].str.split('(',expand = True)
df_norm_name = df_norm_split[0]
df_norm_split = df_norm_split[1].str.split(')',expand = True)
#欠損値の処理を行う。
df_norm_split = df_norm_split.replace('','_')
df_norm_split = df_norm_split.fillna('_')

df_norm_num = df_norm_split[0]
df_norm_exp = df_norm_split[1]

# 分割した文字列を集計する。
df_norm = pd.concat([df_norm,df_norm_name,df_norm_num,df_norm_exp],axis = 1)
df_norm.columns = [0,'R',today,'カード名','型番','エキスパンション']
df_norm = df_norm[['カード名','R','型番','エキスパンション',today]]
# df_norm

In [8]:
#　requestsメソッドを使って、サイトからHTMLの情報を抽出する。
url = 'https://www.fullcomp.jp/radiokaikan/kaitori/19567'
res = requests.get(url)

In [9]:
#　BeautifulSoup()の設定を行う。
soup = BeautifulSoup(res.content,'html.parser')

# 価格情報を抽出する。
soup_data = soup.find('tbody',class_ = 'fs-small')
card_data = soup_data.find_all('td') 

In [10]:
#情報をリストに格納する。
card_rare = []
card_name = []
card_price = []

data_len = len(card_data) - 1
data_len/3

for i in range(int(data_len/3)):
    card_rare.append(card_data[3*i].string)
    card_name.append(card_data[3*i+1].string)
    card_price.append(card_data[3*i+2].string)
    
# データの結合を行う。
df_new = pd.concat([
    pd.DataFrame(card_name),
    pd.DataFrame(card_rare),
    pd.DataFrame(card_price)],axis = 1)

#カラムを設定する。
df_new.columns = [0,1,2]

#文字列の分割を行う。
df_new_split = df_new[0].str.split('【',expand = True)
df_new_split = df_new_split[1].str.split('】',expand = True)
df_new_split = df_new_split[1].str.split('(',expand = True)
df_new_name = df_new_split[0]
df_new_split = df_new_split[1].str.split(')',expand = True)
df_new_num = df_new_split[0]
df_new_exp = df_new_split[1]

# 分割した文字列を集計する。
df_new = pd.concat([df_new,df_new_name,df_new_num,df_new_exp],axis = 1)
df_new.columns = [0,'R',today,'カード名','型番','エキスパンション']
df_new = df_new[['カード名','R','型番','エキスパンション',today]]
# df_new

In [11]:
df_today = pd.concat([df_norm,df_new],ignore_index = True)
df_today.columns = ['カード名','R', '型番','エキスパンション',today]

In [12]:
#集計データを読み込む。
df_preday = pd.read_csv('fullcomp_data.csv')

In [13]:
# df_preday = df_preday.drop(['Unnamed: 0'],axis = 1)

In [14]:
#結合のためのkeyを作る。(df_all)

df_today['key'] = df_today['カード名'].str.cat(df_today['型番'],sep = ' , ',na_rep='')
df_today['key'] = df_today['key'].str.cat(df_today['R'],sep = ' ,' ,na_rep='')
df_today['key'] = df_today['key'].str.cat(df_today['エキスパンション'],sep = ' ,' ,na_rep='')

# print('df_todayのユニークなkeyの個数:',len(df_today['key'].unique()))
# print('df_todayのデータの列数:',len(df_today))
# print('df_todayのkeyに重複のあるデータ:\n',df_today[df_today['key'].duplicated()])

In [15]:
#結合のためのkeyを作る。(df_all)

df_preday['key'] = df_preday['カード名'].str.cat(df_preday['型番'],sep = ' , ',na_rep='')
df_preday['key'] = df_preday['key'].str.cat(df_preday['R'],sep = ' ,' ,na_rep='')
df_preday['key'] = df_preday['key'].str.cat(df_preday['エキスパンション'],sep = ' ,' ,na_rep='')

# print('df_predayのユニークなkeyの個数:',len(df_preday['key'].unique()))
# print('df_predayのデータの列数:',len(df_preday))
# print('df_predayのkeyに重複のあるデータ:\n',df_preday[df_preday['key'].duplicated()])

In [16]:
#keyが被る行にtodayの数値を代入する。

df_preday[today] = 0
df_preday

for i in range(len(df_preday)):
    for j in range(len(df_today)):
        if df_preday.loc[i,'key'] == df_today.loc[j,'key']:
            df_preday.loc[i,today] = df_today.loc[j,today]
            break

In [17]:
count = 0
for i in range(len(df_today)):
    flg = 0
    for j in range(len(df_preday)):
        if df_today.loc[i,'key'] == df_preday.loc[j,'key']:
            flg = 1
            break
            
    if flg == 0:
        df_preday = pd.concat([df_preday,df_today[i:i+1]])
        df_preday.reset_index(drop = True,inplace=True)
        count = count + 1

In [18]:
#データの整理を行う。
df_update = df_preday
df_update = df_update.drop('key',axis = 1)

#データの上書きを行う。
df_update.to_csv('fullcomp_data.csv',index = None)
df_update.to_csv('/Users/kanta/Desktop/Pokemon_work/3.data/2.fullcomp/{}.csv'.format(today),index = None)

## ② データの分析

In [19]:
#数値をint型に変換する。
df_update = df_update.replace(',','', regex=True)
#欠損値の処理を行う。
df_update.fillna({'カード名':'-','R':'-','型番':'-','エキスパンション':'-'},inplace = True)
df_update.fillna(0,inplace = True)

df_update = pd.concat([df_update.iloc[:,:4],df_update.iloc[:,4:].astype(int)],axis = 1)

#前日との値差を計算する。
today = datetime.date.today()
yesterday = today - timedelta(days = 1) 

today = today.strftime('%y-%m-%d')
yesterday = yesterday.strftime('%y-%m-%d')

df_update['前日との値差'] = df_update[today] - df_update[yesterday]  

In [20]:
#値上がりしたカードを抽出する。
print('■ 前日からの値上がりしたカード')
up_num = len(df_update[df_update['前日との値差'] > 0])
pd.set_option('display.max_rows',up_num)
df_update.sort_values('前日との値差',ascending=False)[0:up_num]

■ 前日からの値上がりしたカード


,カード名,R,型番,エキスパンション,21-01-30,...,21-02-06,21-02-07,21-02-08,21-02-09,前日との値差
882,ブラッキー&ダークライGX,SR,182/173,SM12a,6500,...,10000,10000,9000,13000,4000
1167,リザードンVMAX,SSR,308/190,s4a,18000,...,20000,20000,18000,20000,2000
802,ピカチュウVMAX,HR,114/100,s4,20000,...,18000,18000,18000,20000,2000
28,アルセウス&ディアルガ&パルキアGX,SR,100/095,SM12,9500,...,14000,16000,16000,17500,1500
152,エーフィ&デオキシスGX,SR,177/173,SM12a,2400,...,3000,3000,3000,4000,1000
73,イーブイVMAX,P,101/S-P,PROMO,3400,...,3000,3500,3500,4500,1000
1409,ブルーの探索,SR,061/054,SM9b,7000,...,7000,8000,8000,9000,1000
1411,ブルーの探索,SR,196/173,SM12a,7500,...,7500,8000,8000,9000,1000
427,サーナイト&ニンフィアGX,SR,061/055,SM9a,6000,...,8500,9000,9000,10000,1000
706,ニンフィアGX,P,323/SM-P,PROMO,8000,...,8000,8000,8000,9000,1000


In [21]:
print('■ 前日からの値下がりが大きいカード')
down_num = len(df_update[df_update['前日との値差'] < 0])
pd.set_option('display.max_rows',down_num)
df_update.sort_values('前日との値差',ascending=True).head(down_num)

■ 前日からの値下がりが大きいカード


,カード名,R,型番,エキスパンション,21-01-30,...,21-02-06,21-02-07,21-02-08,21-02-09,前日との値差
1463,マリィ,SR,198/190,s4a,25000,...,32000,32000,30000,28000,-2000
1166,リザードンV,SSR,307/190,s4a,10000,...,10000,10000,10000,9000,-1000
1960,いちげきウーラオスVMAX,HR,085/070,s5I,16500,...,16000,16000,15000,14000,-1000
2000,れんげきウーラオスVMAX,HR,084/070,s5R,16500,...,16000,16000,16000,15000,-1000
2037,いちげきエネルギー,UR,091/070,s5I,6000,...,5600,5600,5000,4500,-500
361,ゲッコウガGX,SSR,216/150,SM8b,3000,...,3000,3200,3200,2800,-400
10,アーマードミュウツー,P,365/SM-P,PROMO,800,...,800,800,800,500,-300
2045,れんげきエネルギー,UR,091/070,s5R,4650,...,4000,4000,3800,3600,-200
1958,いちげきウーラオスV,SR,075/070,s5I,3500,...,3200,3200,3200,3000,-200
1173,リザードン&テールナーGX,HR,075/064,SM11a,3200,...,3800,3650,3800,3650,-150


In [22]:
#新規に値段がついたカードを表示する。
print('■ 新規にデータを追記したカード')
df_update.tail(count)

■ 新規にデータを追記したカード


,カード名,R,型番,エキスパンション,21-01-30,...,21-02-06,21-02-07,21-02-08,21-02-09,前日との値差
